In [3]:
from getpass import getpass

api_key_prompt = "Enter OpenAI API key:" 
api_key = getpass(api_key_prompt)

In [61]:
from haystack.document_stores import ElasticsearchDocumentStore

es_document_store = ElasticsearchDocumentStore(index="documents", embedding_dim=1536)

In [6]:
!pip install datasets

from datasets import load_dataset

remote_dataset = load_dataset('Tuana/pycon-usa-2023', split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/131 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /Users/tuanacelik/.cache/huggingface/datasets/Tuana___parquet/Tuana--pycon-usa-2023-76481837ef253f5c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [295]:
from haystack.schema import Document

documents = documents = [Document.from_dict(document) for document in remote_dataset]
# for document in documents:
#     document.content = document.content.replace('\n', '<br />')

es_document_store.write_documents(documents)

In [294]:
es_document_store.delete_documents()

/Users/tuanacelik/opt/anaconda3/envs/jupyter/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [296]:
es_document_store.get_all_documents()[0]

<Document: {'content': 'Talks: How we are making CPython faster. Past, present and future.\nSaturday - April 22nd, 2023 2:30 p.m.-3 p.m. in\nPresented by:\nMark Shannon\n\nExperience Level:\nSome experience\nDescription\nMany of you will will have heard that Python 3.11 is considerably faster than 3.10.\nHow did we do that? How are we going to make 3.12 and following releases even faster?\nIn this talk, I will present a high level overview of the approach we are taking to speeding up CPython.\nStarting with a simple overview of some basic principles, I will show how we can apply those to streamline and speedup CPython.\nI will try to avoid computer science and software engineering terminology, in favor of diagrams, a few simple examples, and some high-school math.\nFinally, I make some estimates about how much faster the next few releases of CPython will be, and how much faster Python could go..', 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0, 'url': 'https://us.pycon.

In [398]:
from haystack.nodes import AnswerParser, PromptNode, PromptTemplate
from haystack.agents import Agent, Tool

schedule_result = PromptTemplate(
            name="zero-shot-react-schedule", 
            prompt_text="You are a helpful and knowledgeable agent for PyCon attendees. To achieve your goal of creating an itinerary "
                        "correctly that takes date, speaker and topic restrictions into consideration, you have access to the following tools:\n\n"
                        "{tool_names_with_descriptions}\n\n"
                        "To answer questions, you'll need to go through multiple steps involving step-by-step thinking and "
                        "selecting appropriate tools and their inputs; tools will respond with observations."
                        "When you are ready wity an itinerary, respond with the `Itinerary:`\n\n"
                        "Use the following format:\n\n"
                        "Question: the question to be answered\n"
                        "Thought: Reason if you have the final itinerary. If yes, answer the query. If not, continue using the tools to create an itinerery.\n"
                        "Tool: pick one of {tool_names} \n"
                        "Tool Input: the input for the tool.\n"
                        "Observation: the tool will respond with the result\n"
                        "...\n"
                        "Itinerary: the final itinrary in chronological order with up to 5 events, where each line is one event with the time, title and speaker of the event.\n\n"
                        "Thought, Tool, Tool Input, and Observation steps can be repeated multiple times, but sometimes we can find an answer in the first pass\n"
                        "---\n\n"
                        "Question: {query}\n"
                        "Thought: Let's think step-by-step, I first need to ",
        )

prompt_node = PromptNode(model_name_or_path="text-davinci-003", api_key=api_key, stop_words=["Observation:"], max_length=1000)
agent = Agent(prompt_node=prompt_node, prompt_template=schedule_result, final_answer_pattern="Itinerary\s*:\s*(.*)")

In [338]:
from haystack.nodes import BM25Retriever, EmbeddingRetriever

embedding_retriever = EmbeddingRetriever(document_store= es_document_store,
                                embedding_model='text-embedding-ada-002', 
                                batch_size = 32, 
                                api_key=api_key,
                                top_k=7)

es_document_store.update_embeddings(embedding_retriever)


Updating embeddings:   0%|          | 0/131 [00:00<?, ? Docs/s]

Calculating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

The prompt has been truncated from 522 tokens to 512 tokens to fit within the max token limit. Reduce the length of the prompt to prevent it from being cut off.
The prompt has been truncated from 659 tokens to 512 tokens to fit within the max token limit. Reduce the length of the prompt to prevent it from being cut off.


In [339]:
bm25_retriever = BM25Retriever(document_store=es_document_store)

In [310]:

from haystack.nodes import AnswerParser

# list_events = PromptTemplate(name="list_events", prompt_text="""You will be provided a list of events such as 'Talks', 'Sponsor Presentations' or 'Tutorials'. 
#                                                                 They will appear in the following format: \n
                                                                
#                                                                 Events:
#                                                                 Type of event: Title of event
#                                                                 Day of week - April date, time
#                                                                 Presented by
#                                                                 Names of presenters
#                                                                 Description
#                                                                 description of event

#                                                                 Provide an itinerey with the date, title and short summary of each event.
#                                                                 Filter the events based on the query where necessary and the order should 
#                                                                 be based on day of week, date and time (earliest to latest)
                                                                
#                                                                 Query: {query};
#                                                                 Events: {join(documents)};
#                                                                 Answer: 
#                                                                 """, 
                                                                
#                                                                 output_parser=AnswerParser(),)
list_events = PromptTemplate(name="list_events", prompt_text="""You will be provided a list of events, such as 'Talks', 'Sponsor Presentations', or 'Tutorials', in the format below:

                                                                Events:
                                                                Type of event: Title of event
                                                                Day of week - April date, time
                                                                Presented by
                                                                Names of presenters
                                                                Description
                                                                description of event

                                                                Create an itinerary for the attendee by summarizing each event in a single line. 
                                                                Filter the events based on the given query. 
                                                                Arrange the events in chronological order based on the day of the week, date, and time.

                                                                Query: {query};
                                                                Events: {join(documents)};
                                                                Answer:
                                                                """, 
                                                                
                                                                output_parser=AnswerParser(),)

list_builder = PromptNode(model_name_or_path="text-davinci-003", api_key=api_key, max_length=300, default_prompt_template=list_events)

In [311]:
from haystack import Pipeline

event_list_by_topic_pipeline = Pipeline()
event_list_by_topic_pipeline.add_node(component=embedding_retriever, name="Retriever", inputs=["Query"])
event_list_by_topic_pipeline.add_node(component=list_builder, name="TopicLister", inputs=["Retriever"])

In [312]:
result = event_list_by_topic_pipeline.run("I'm interested in NLP, what should I see?")

Calculating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/tuanacelik/opt/anaconda3/envs/jupyter/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
1 out of the 1 completions have been truncated before reaching a natural stopping point. Increase the max_tokens parameter to allow for longer completions.


In [313]:
from haystack.utils import print_answers

print_answers(result, details="minimum")

"Query: I'm interested in NLP, what should I see?"
'Answers:'
[   {   'answer': 'Events (NLP): \n'
                  '\n'
                  'Tutorials: Intro to Hugging Face: Fine-tuning BERT for NLP '
                  'tasks (Wednesday - April 19th, 2023 9 a.m.-12:30 p.m.)\n'
                  'Sponsor Presentations: Building LLM-based Agents: How to '
                  'develop smart NLP-driven apps with Haystack (Thursday - '
                  'April 20th, 2023 1:30 p.m.-2:30 p.m.)\n'
                  'Talks: Approaches to Fairness and Bias Mitigation in '
                  'Natural Language Processing (Friday - April 21st, 2023 '
                  '10:45 a.m.-11:15 a.m.)\n'
                  'Talks: Start thinking small: Next level Machine Learning '
                  'with TinyML and Python (Friday - April 21st, 2023 2:30 '
                  'p.m.-3 p.m.)\n'
                  'Charlas: Resolviendo crimenes con Python mediante el '
                  'Procesamiento del Lenguaje Na

In [293]:
print(result["answers"][0].answer.replace("<br />", "\n"))

either of them, as the tutorial will start from scratch. The tutorial will also be useful for experienced users, as it will show different usages and tricks to be used in the development of their projects.

Itinerary: 
Tutorials: Intro to Hugging Face: Fine-tuning BERT for NLP tasks
Wednesday - April 19th, 2023 9 a.m.-12:30 p.m. in
Presented by:
Juhi Chandalia
Dana Engebretson

Experience Level:
Advance experience
Description
You’ve heard about ChatGPT’s conversational ability and how DALL-E can create images from a simple phrase. Now, you want to get your hands dirty training some state of the art (SOTA) deep learning models. We will use Jupyter notebooks to fine-tune an NLP model based on BERT to do sentiment analysis.
In this hands-on tutorial, we will learn about using HuggingFace models from pre-trained open-source checkpoints and adapting these models to our own specific tasks. We will see that using SOTA NLP and computer vision models has been made easier with a combination of H

In [314]:
event_list_by_keyword_pipeline = Pipeline()
event_list_by_keyword_pipeline.add_node(component=bm25_retriever, name="Retriever", inputs=["Query"])
event_list_by_keyword_pipeline.add_node(component=list_builder, name="TopicLister", inputs=["Retriever"])

In [315]:
result = event_list_by_keyword_pipeline.run("Tuana")

In [316]:
print_answers(result, details="minimum")

'Query: Tuana'
'Answers:'
[   {   'answer': 'Thursday - April 20th, 2023 1:30 p.m.-2:30 p.m.: Sponsor '
                  'Presentations: Building LLM-based Agents: How to develop '
                  'smart NLP-driven apps with Haystack (Sponsor: deepset) '
                  'presented by Tuana Celik'}]


In [399]:
date_search = Tool(name="DateSearch", pipeline_or_node=event_list_by_keyword_pipeline, 
                           description="Useful for when you want to filter events based on date.", 
                           output_variable="answers")

speaker_search = Tool(name="SpeakerSearch", pipeline_or_node=event_list_by_keyword_pipeline, 
                           description="Useful for when you want to filter events based on speaker.", 
                           output_variable="answers")

topic_search = Tool(name="TopicSearch", pipeline_or_node=event_list_by_topic_pipeline, 
                         description="Useful for when you need to find events based on what the attendee is interested in.", 
                         output_variable="answers")

In [400]:
agent.add_tool(date_search)
agent.add_tool(speaker_search)
agent.add_tool(topic_search)

In [403]:
agent.run("I want to learn about NLP")


Agent zero-shot-react-schedule started with {'query': 'I want to learn about NLP', 'params': None}
 use TopicSearch
Tool: TopicSearch
Tool Input: NLP


Calculating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

The prompt has been truncated from 3927 tokens to 3797 tokens so that the prompt length and answer length (300 tokens) fit within the max token limit (4097 tokens). Reduce the length of the prompt to prevent it from being cut off.
1 out of the 1 completions have been truncated before reaching a natural stopping point. Increase the max_tokens parameter to allow for longer completions.


Observation: tareas, como la automatización de procesos de extracción de información que impactan directamente en el desarrollo de proyectos con tecnología de visión artificial.

Itinerary: 
Talks: Resolviendo crimenes con Python mediante el Procesamiento del Lenguaje Natural (NLP) - Saturday - April 22nd, 2023 2:30 p.m.-3 p.m.
Talks: Approaches to Fairness and Bias Mitigation in Natural Language Processing - Friday - April 21st, 2023 10:45 a.m.-11:15 a.m.
Sponsor Presentations: Building LLM-based Agents: How to develop smart NLP-driven apps with Haystack (Sponsor: deepset) - Thursday - April 20th, 2023 1:30 p.m.-2:30 p.m.
Tutorials: Intro to Hugging Face: Fine-tuning BERT for NLP tasks - Wednesday - April 19th, 2023 9 a.m.-12:30 p.m.
Tutorials: Going beyond with Jupyter Notebooks: Write your first package using literate programming - Wednesday - April 19th, 2023 9 a.m.-12:30 p.m.
Talks: Skyn
Thought:  Let's see if the attendee is interested in a specific speaker
Tool: SpeakerSearch
To

Exception: Exception while running node 'TopicLister': Expected prompt parameters ['documents', 'query'] but got ['stop_words', 'query', 'top_k'].
Enable debug logging to see the data that was passed when the pipeline failed.